# AI Tools

## Project - Airlines AI Assistant

We will make an AI Customer Support assistant for an Airline with pre-fixed prices

In [1]:
# Importing the Libraries

import requests
import json
import time
import re
import ollama
from typing import List
from IPython.display import Markdown, display, update_display
from bs4 import BeautifulSoup
import gradio as gr

In [2]:
# Initializing System Messages and ollama details

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

# system_message += """
# TOOLS INSTRUCTIONS:
# - Only call a tool if the user explicitly asks about flight booking, ticket prices, or related information.
# - If the user is greeting, chatting, or asking a general question, DO NOT call a tool. Just respond normally.
# """

OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME="qwen3"

In [3]:
# Chat function for gradio

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    payload = {
        "model": MODEL_NAME,
        "messages": messages,
        "temperature": 0.8,
        "stream": True  # Important: to get streaming responses
    }
    start_time = time.time()
    response = requests.post(OLLAMA_URL, json=payload, stream=True)
    result=""
    try:
        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line.decode('utf-8'))
                    delta = data.get("message", {}).get("content", "")
                    if delta:
                        # Optional: remove unwanted <think> tags or others
                        clean_delta = re.sub(r"<think>.*?</think>", "", delta, flags=re.DOTALL)
                        result += clean_delta
                        yield result
                except json.JSONDecodeError:
                    continue
    finally:
        # Ensure generator exits cleanly
        yield result

In [4]:
# Gradio UI

gr.ChatInterface(fn=chat, chatbot=gr.Chatbot(type="messages")).launch()

/home/alexender/Desktop/Projects/My_projects/envs/Torch_env/lib/python3.12/site-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


### Tools
Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

In [40]:
# Let's start by making a useful function

ticket_prices = {"chennai": "₹ 399", "mumbai": "₹ 899", "bengalore": "₹ 400", "hydrabad": "₹ 499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "")

In [41]:
get_ticket_price("Chennai")

Tool get_ticket_price called for Chennai


'₹ 399'

In [42]:
# There's a particular dictionary structure that's required to describe our function:

# Tool Name
price_function = {
        "name": "get_ticket_price",  # Name of the function
        # This is important since its passed to LLM so that when it should call the tool we have to mention it clearly, its like system prompt for tools. Be clear as possible 
        "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
        "parameters": {
            # Here we have to give the details of the parameters, here we have one parameter so mentioning that here
            "type": "object",
            "properties":{
                "destination_city":{
                    # Mentioning the data type and where and how this input is used
                    "type": "string",
                    "description": "The city that the customer wants to travel to",
                },
            },
            # we have to mention what are the must required parameters of the function(tool)
            "required":["destination_city"],
            # It restricts the input object to allow only the explicitly defined properties — no extras.
            # like only "destination_city": "Berlin" this is allowed they can't send extra prameters like "class": "economy"
            "additionalProperties": False,
        }
    }

In [43]:
# And this is included in a list of tools:

tools = [
    {
        "type": "function",  # Since we are calling the function we have to mention it
        "function": price_function,
    }
]

In [44]:
# We have to write that function handle_tool_call:

import uuid
def handle_tool_call(message):
    tool_call = message["tool_calls"][0] # Since we are using only one tool we are getting the first one
    arguments = tool_call["function"]["arguments"]
    if isinstance(arguments, str):
        arguments = json.loads(arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)

    tool_call_id = tool_call.get("id", str(uuid.uuid4()))
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city, "price": price}),
        "tool_call_id": tool_call_id
    }
    return response, city

In [45]:
# For tools we are going to use ollama package instead of APIs(Since APIs don't support tools)
# This tools support will be provides by only specific models so we switch to Qwen from Mistral

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    result = ""
    try:
        response = ollama.chat(
            model=MODEL_NAME,
            messages=messages,
            tools=tools,
            stream=False,
            options={
                "temperature": 0.8
            }
        )
        print(response)

        msg = response.get("message", {})
        if msg.get("tool_calls"):
            print(msg.get("tool_calls"))
            tool_response, city = handle_tool_call(msg)
            messages.append(msg)
            messages.append(tool_response)
            response = ollama.chat(
                model=MODEL_NAME,
                messages=messages,
                stream=False,
                options={
                    "temperature": 0.8
                }
            )

            msg = response.get("message", {})
            
        if msg.get("content"):
            return {"role": "assistant", "content": msg["content"]}
        
        return {"role": "assistant", "content": "I'm not sure how to respond."}
                    
    except Exception as e:
        print("Exception: ", repr(e))

In [46]:
# Chat Interface

gr.ChatInterface(fn=chat, chatbot=gr.Chatbot(type="messages")).launch()

/home/alexender/Desktop/Projects/My_projects/envs/Torch_env/lib/python3.12/site-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


model='qwen3' created_at='2025-08-30T16:32:37.815462099Z' done=True done_reason='stop' total_duration=15067125375 load_duration=51053623 prompt_eval_count=224 prompt_eval_duration=2581384020 eval_count=70 eval_duration=12428458945 message=Message(role='assistant', content='<think>\nOkay, the user said "hello". I need to respond politely and briefly. Since there\'s no specific query here, I should just greet them back and offer assistance. No need to call any functions because they didn\'t ask for anything yet. Keep it friendly and open-ended.\n</think>\n\nHello! How can I assist you today?', thinking=None, images=None, tool_calls=None)
